<a href="https://colab.research.google.com/github/tthogho1/CompareImage/blob/main/007_stable_diffusion_gui_basic_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion GUI Basic 2.0
Stable Diffusion 2.0 easy and useful GUI basic tool

GitHub repository is below:
[stable-diffusion-colab-tools](https://github.com/karaage0703/stable-diffusion-colab-tools)

In [ ]:
#@title **Launch App**
#@markdown　Execute and click URL ex: `Running on public URL: Running on https://localhost:7860/` 


!pip -qq install diffusers[torch]==0.11.1 transformers
!pip -qq install --upgrade --pre triton

# install xformers
# https://github.com/XavierXiao/Dreambooth-Stable-Diffusion/issues/102
# !pip -qq install https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
!pip -qq install https://github.com/metrolobo/xformers_wheels/releases/download/4c06c79_various6/xformers-0.0.15.dev0_4c06c79.d20221201-cp38-cp38-linux_x86_64.whl

!pip -qq install transformers scipy ftfy gradio
!pip -qq install "ipywidgets>=7,<8"

from google.colab import output
output.enable_custom_widget_manager()

import gradio as gr
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

#@markdown　Select model 

device = "cuda"
model_id = "stabilityai/stable-diffusion-2"

if model_id == "stabilityai/stable-diffusion-2":
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id, 
        scheduler=EulerDiscreteScheduler.from_pretrained(
        model_id, 
        subfolder="scheduler"
        ), 
        torch_dtype=torch.float16,
        revision="fp16"
    ).to("cuda")
    pipe.enable_attention_slicing()


def infer(prompt, num_images, num_inference_steps, guidance_scale_value, width_images, height_images, seed_number):
    generator = torch.Generator(device=device)
    latents = None
    seeds = []

    width = int(width_images)
    height = int(height_images)
    num_images = int(num_images)
    num_inference_steps = int(num_inference_steps)
    seed_number = int(seed_number)

    images = []

    for _ in range(num_images):
        # Get a new random seed, store it and use it as the generator state
        if seed_number < 0:
            seed = generator.seed()
        else:
            seed = seed_number

        print('seed=' + str(seed))
        seeds.append(seed)
        generator = generator.manual_seed(seed)
    
        image_latents = torch.randn(
            (1, pipe.unet.in_channels, height // 8, width // 8),
            generator = generator,
            device = device
        )
        latents = image_latents if latents is None else torch.cat((latents, image_latents))

    for latent in latents:
        with torch.autocast('cuda'):
            image = pipe(
                [prompt],
                width=width,
                height=height,
                guidance_scale=guidance_scale_value,
                num_inference_steps=num_inference_steps,
                latents = latent.unsqueeze(dim=0)
            ).images[0]
        images.append(image)

    return images


from IPython.display import clear_output

block = gr.Blocks(css=".container { max-width: 800px; margin: auto; }")

with block as demo:
    gr.Markdown("<h1><center>Stable Diffusion Tool</center></h1>")
    gr.Markdown(
        'Stable Diffusion useful web tool'
    )
    with gr.Group():
        with gr.Box():
            gr.Markdown(
                'Enter prompt and Run!!'
            )
            with gr.Row().style(mobile_collapse=False, equal_height=True):

                text = gr.Textbox(
                    label='Enter prompt', show_label=False, max_lines=1
                ).style(
                    border=(True, False, True, True),
                    rounded=(True, False, False, True),
                    container=False,
                )
                btn = gr.Button("Run").style(
                    margin=False,
                    rounded=(False, True, True, False),
                )

        num_images = gr.Number(
                    label='Number of images', value=1
                )

        seed_number = gr.Number(
                    label='Seed(-1 is random)', value=-1
                )

        num_inference_steps = gr.Slider(
                    label='Number of inference steps', minimum=1, maximum=200, value=20
                )

        guidance_scale_value = gr.Slider(
                    label='Guidance scale', minimum=1, maximum=20, value=7.5, step=0.1
                )

        width_images = gr.Slider(
                    label='Width of images', minimum=64, maximum=1024, value=768, step=64
                )

        height_images = gr.Slider(
                    label='Height of images', minimum=64, maximum=1024, value=768, step=64
                )

        gallery = gr.Gallery(label="Generated images", show_label=False).style(
            grid=[2], height="auto"
        )

        btn.click(infer,
                 inputs=[text, num_images, num_inference_steps, guidance_scale_value, width_images, height_images, seed_number], outputs=gallery)

    gr.Markdown(
        """___
   <p style='text-align: center'>
   Created by CompVis and Stability AI
   <br/>
   </p>"""
    )

clear_output()
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

seed=7108470459065770


  0%|          | 0/20 [00:00<?, ?it/s]

seed=2528300416135270


...

  0%|          | 0/20 [00:00<?, ?it/s]

seed=1067712464344051


  0%|          | 0/20 [00:00<?, ?it/s]

seed=2321567230011004


  0%|          | 0/20 [00:00<?, ?it/s]

## Reference
Special Thanks
- https://note.com/npaka/n/n5fd3d8ecf1e6
- https://note.com/npaka/n/ndd549d2ce556
- http://cedro3.com/ai/image2image/
- https://colab.research.google.com/github/pcuenca/diffusers-examples/blob/main/notebooks/stable-diffusion-seeds.ipynb
- https://huggingface.co/hakurei/waifu-diffusion
- https://huggingface.co/naclbit/trinart_stable_diffusion_v2